# Battle of the Neighborhoods
### Introduction

This project is for who is planning to open a Coffee House with fast food in Ankara, Turkey. This project suggests the best locations for Coffee Houses in Ankara. Ankara is the capital of Turkey with a population of 5M. Turkish culture has deep roots in coffee. Also, it is getting popular because of its affordable prices. Coffee consumption has increased 13% and average consumption is 1.1 kg/person in 2018. This report explores which neighborhoods of Ankara have the most as well as the best Coffee Houses. Also, this project answers the questions “Where should I open an Coffee House?” and “Where should I stay If I want a tasty coffee?”


### Problem Description:
A Cofee restaurant is a business which prepares and serves fastfood and  drink to customers in return for money, either paid before the meal, after the meal, or with an open account. The City of Ankara is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

Central and Turkeys immigrants, especially Europians Turists -wurst tost, bagels, cheesecake, hot dog, knishes, and delicatessens
Italian turists - Ankara-style pizza and Italian cuisine
Chinese and other Asian restaurants, sandwich joints, trattorias, diners, and coffeehouses are ubiquitous throughout the city
Middle Eastern foods such as falafel and kebabs examples of modern Ankara street food
It is famous for not just Cafe's but also for fine dining  restaurants.The city is home to "nearly one thousand of the finest and most diverse haute cuisine restaurants in the world".

## Data
District of Ankara are obtained from https://en.wikipedia.org/wiki/Ankara_Province
Latitude and Longitude values are obtained by using "geocoder".
All data related to locations will be obtaine by using FourSquare API and Python Libraries.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import geocoder

In [3]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
wiki_page = requests.get(wiki_link)
wiki_page

<Response [200]>

In [4]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [5]:
rows = table.find_all('tr')

In [6]:

columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [7]:
df_ankara = pd.DataFrame(columns = columns)

In [8]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [9]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [10]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [12]:

df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [13]:
df_ankara_loc.head()

,District,Population (2017),Area (km²),Density (per km²),Latitude,Longitude
0,Akyurt,32.863,369,89,40.13082,33.08719
1,Altındağ,371.366,123,3.019,39.94171,32.85445
2,Ayaş,12.289,1.041,12,40.01516,32.33270
3,Bala,21.682,1.851,12,39.55391,33.12352
4,Beypazarı,48.476,1.697,29,40.16811,31.92052


In [14]:
import numpy as np
import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium


In [15]:
from geopy.geocoders import Nominatim 

address = "Çankaya, Ankara"

geolocator = Nominatim(user_agent = "Ankara_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Ankara are {}, {}.".format(latitude, longitude))

The geographical coordinates of Ankara are 39.9207893, 32.8540412.


In [16]:
map_ankara = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_ankara_loc["Latitude"], df_ankara_loc["Longitude"], df_ankara_loc["District"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_ankara)  
    
map_ankara

In [19]:
CLIENT_ID = "***************"
CLIENT_SECRET = "**********************" 
VERSION = "20180605"

In [18]:
df_ankara_loc.loc [0, "District"]
df_ankara_loc.loc [0, "District"]

'Akyurt'

In [17]:
neighborhood_latitude = df_ankara_loc.loc[0, "Latitude"]
neighborhood_longitude = df_ankara_loc.loc[0, "Longitude"] 

neighborhood_name = df_ankara_loc.loc[0, "District"] 

print("Latitude and longitude values of the neighborhood {} are {}, {}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of the neighborhood Akyurt are 40.13082000000003, 33.08719000000008.


In [21]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [22]:

results = requests.get(url).json()


In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

<ipython-input-24-f5e18700a056>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Meşhur Köfteci Bodur,Turkish Restaurant,40.130719,33.086776
1,Akyurt kültür parkı,Park,40.133123,33.085071
2,Yurtalan Restaurant,Kebab Restaurant,40.132672,33.084306
3,Beyazıt Sofrası,Steakhouse,40.130542,33.081901
4,Onur pide ve kebap salonu,Kebab Restaurant,40.130807,33.086422
5,Yurtalan,Turkish Restaurant,40.132743,33.084385
6,şişman pasta&cafe,Café,40.130820,33.085025
7,Yurtalan Restaurant,Doner Restaurant,40.132641,33.084492
8,Akyurt Sofrası,Turkish Restaurant,40.130854,33.086709
9,Akyurt,Farm,40.131012,33.083330


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
ankara_venues = getNearbyVenues(names=df_ankara_loc['District'],
                                   latitudes=df_ankara_loc['Latitude'],
                                   longitudes=df_ankara_loc['Longitude']
                                  )

Akyurt
Altındağ
Ayaş
Bala
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kahramankazan
Kalecik
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle
Urban (9 districts)Altındağ, Çankaya, Etimesgut, Gölbaşı, Keçiören, Mamak, Pursaklar, Sincan, Yenimahalle
TOTAL


In [27]:

ankara_venues.groupby("District").count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Akyurt,31,31,31,31,31,31
Altındağ,100,100,100,100,100,100
Ayaş,33,33,33,33,33,33
Bala,11,11,11,11,11,11
Beypazarı,67,67,67,67,67,67
Elmadağ,52,52,52,52,52,52
Etimesgut,100,100,100,100,100,100
Evren,5,5,5,5,5,5
Gölbaşı,100,100,100,100,100,100


In [28]:
print('There are {} unique venue categories.'.format(len(ankara_venues['Venue Category'].unique())))

There are 232 unique venue categories.


In [29]:

ankara_onehot = pd.get_dummies(ankara_venues[['Venue Category']], prefix="", prefix_sep="")

ankara_onehot['Neighborhood'] = ankara_venues['District'] 

fixed_columns = [ankara_onehot.columns[-1]] + list(ankara_onehot.columns[:-1])
ankara_onehot = ankara_onehot[fixed_columns]

ankara_onehot

,Neighborhood,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Wings Joint,Women's Store,Yoga Studio
0,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Akyurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1669,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1670,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1671,TOTAL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:

ankara_grouped = ankara_onehot.groupby('Neighborhood').mean().reset_index()
ankara_grouped

,Neighborhood,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,Airport Lounge,Antique Shop,Aquarium,Arcade,Art Gallery,...,Used Bookstore,Video Game Store,Vineyard,Volleyball Court,Water Park,Waterfront,Wedding Hall,Wings Joint,Women's Store,Yoga Studio
0,Akyurt,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00
1,Altındağ,0.000000,0.00,0.00,0.00,0.00000,0.03,0.000000,0.010000,0.03,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00
2,Ayaş,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00
3,Bala,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00
4,Beypazarı,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.014925,0.00,...,0.00,0.000000,0.0000,0.014925,0.000000,0.0,0.000000,0.00,0.000000,0.00
5,Elmadağ,0.019231,0.00,0.00,0.00,0.00000,0.00,0.000000,0.019231,0.00,...,0.00,0.000000,0.0000,0.000000,0.019231,0.0,0.000000,0.00,0.000000,0.00
6,Etimesgut,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.040000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.01,0.000000,0.01
7,Evren,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.2,0.000000,0.00,0.000000,0.00
8,Gölbaşı,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.040000,0.00,...,0.00,0.000000,0.0000,0.000000,0.010000,0.0,0.000000,0.00,0.000000,0.00
9,Güdül,0.000000,0.00,0.00,0.00,0.00000,0.00,0.000000,0.000000,0.00,...,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00


In [31]:

num_top_venues = 10

for hood in ankara_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ankara_grouped[ankara_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akyurt----
                venue  freq
0                Farm  0.13
1  Turkish Restaurant  0.10
2    Kebab Restaurant  0.06
3                Park  0.06
4                Café  0.06
5         Music Venue  0.03
6  Basketball Stadium  0.03
7            Tea Room  0.03
8       Big Box Store  0.03
9            Mountain  0.03


----Altındağ----
                venue  freq
0                Café  0.10
1      History Museum  0.07
2             Theater  0.05
3  Turkish Restaurant  0.05
4               Hotel  0.04
5       Historic Site  0.04
6       Jewelry Store  0.04
7          Steakhouse  0.03
8    Kebab Restaurant  0.03
9          Restaurant  0.03


----Ayaş----
                     venue  freq
0       Turkish Restaurant  0.15
1         Kebab Restaurant  0.06
2                     Lake  0.06
3                   Buffet  0.06
4                     Park  0.06
5               Restaurant  0.06
6  Comfort Food Restaurant  0.03
7              Coffee Shop  0.03
8               Food Court  0.03
9    

In [32]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
ankara_venues_sorted = pd.DataFrame(columns=columns)
ankara_venues_sorted['Neighborhood'] = ankara_grouped['Neighborhood']

for ind in np.arange(ankara_grouped.shape[0]):
    ankara_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ankara_grouped.iloc[ind, :], num_top_venues)

ankara_venues_sorted.head(27)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Kebab Restaurant,Café,Park,Pharmacy,Basketball Court,Stadium,Tea Room,Furniture / Home Store
1,Altındağ,Café,History Museum,Theater,Turkish Restaurant,Jewelry Store,Historic Site,Hotel,Kebab Restaurant,Antique Shop,Art Gallery
2,Ayaş,Turkish Restaurant,Buffet,Park,Lake,Kebab Restaurant,Restaurant,Food Court,Café,Snack Place,Farm
3,Bala,Electronics Store,Mountain,Nature Preserve,Business Service,Big Box Store,Convenience Store,Park,Bakery,Café,Turkish Restaurant
4,Beypazarı,Turkish Restaurant,History Museum,Bakery,Tea Room,Scenic Lookout,Hotel,Restaurant,Motel,Fast Food Restaurant,Bistro
5,Elmadağ,Café,Park,Campground,Dessert Shop,Convenience Store,Kebab Restaurant,Bar,Big Box Store,Seafood Restaurant,Bus Stop
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Steakhouse,Shopping Mall,Arcade,Dessert Shop,Park,Bakery,Snack Place
7,Evren,Park,River,Waterfront,Grocery Store,Steakhouse,Farmers Market,Entertainment Service,Event Space,Farm,Fish & Chips Shop
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Coffee Shop,Arcade,Middle Eastern Restaurant,Dessert Shop,Bakery
9,Güdül,Convenience Store,Turkish Restaurant,Botanical Garden,Moving Target,River,Castle,Plaza,Pharmacy,Department Store,Diner


In [34]:
from sklearn.cluster import KMeans

In [38]:
ks = 3

ankara_grouped_clustering = ankara_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=ks, random_state=0).fit(ankara_grouped_clustering)

kmeans.labels_[1:10]

array([1, 1, 1, 1, 1, 1, 0, 1, 2], dtype=int32)

In [40]:
ankara_merged = df_ankara
ankara_merged['Cluster Labels'] = kmeans.labels_

ankara_merged = ankara_merged.join(ankara_venues_sorted.set_index('Neighborhood'), on='District')
ankara_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,Farm,Turkish Restaurant,Kebab Restaurant,Café,Park,Pharmacy,Basketball Court,Stadium,Tea Room,Furniture / Home Store
1,Altındağ,Café,History Museum,Theater,Turkish Restaurant,Jewelry Store,Historic Site,Hotel,Kebab Restaurant,Antique Shop,Art Gallery
2,Ayaş,Turkish Restaurant,Buffet,Park,Lake,Kebab Restaurant,Restaurant,Food Court,Café,Snack Place,Farm
3,Bala,Electronics Store,Mountain,Nature Preserve,Business Service,Big Box Store,Convenience Store,Park,Bakery,Café,Turkish Restaurant
4,Beypazarı,Turkish Restaurant,History Museum,Bakery,Tea Room,Scenic Lookout,Hotel,Restaurant,Motel,Fast Food Restaurant,Bistro
5,Elmadağ,Café,Park,Campground,Dessert Shop,Convenience Store,Kebab Restaurant,Bar,Big Box Store,Seafood Restaurant,Bus Stop
6,Etimesgut,Turkish Restaurant,Café,Restaurant,Steakhouse,Shopping Mall,Arcade,Dessert Shop,Park,Bakery,Snack Place
7,Evren,Park,River,Waterfront,Grocery Store,Steakhouse,Farmers Market,Entertainment Service,Event Space,Farm,Fish & Chips Shop
8,Gölbaşı,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Coffee Shop,Arcade,Middle Eastern Restaurant,Dessert Shop,Bakery
9,Güdül,Convenience Store,Turkish Restaurant,Botanical Garden,Moving Target,River,Castle,Plaza,Pharmacy,Department Store,Diner


In [41]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(ks)
ys = [i+x+(i*x)**2 for i in range(ks)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ankara_merged['Latitude'], ankara_merged['Longitude'], ankara_merged['District'], ankara_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster0

In [42]:

ankara_merged.loc[ankara_merged['Cluster Labels'] == 0, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Çubuk,40.23891,33.03196,0,Café,Turkish Restaurant,Convenience Store,Bistro,Park,Gym / Fitness Center,Mobile Phone Shop,Diner,Fast Food Restaurant,Restaurant


### Cluster1

In [43]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 1, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akyurt,40.13082,33.08719,1,Farm,Turkish Restaurant,Kebab Restaurant,Café,Park,Pharmacy,Basketball Court,Stadium,Tea Room,Furniture / Home Store
1,Altındağ,39.94171,32.85445,1,Café,History Museum,Theater,Turkish Restaurant,Jewelry Store,Historic Site,Hotel,Kebab Restaurant,Antique Shop,Art Gallery
2,Ayaş,40.01516,32.33270,1,Turkish Restaurant,Buffet,Park,Lake,Kebab Restaurant,Restaurant,Food Court,Café,Snack Place,Farm
3,Bala,39.55391,33.12352,1,Electronics Store,Mountain,Nature Preserve,Business Service,Big Box Store,Convenience Store,Park,Bakery,Café,Turkish Restaurant
4,Beypazarı,40.16811,31.92052,1,Turkish Restaurant,History Museum,Bakery,Tea Room,Scenic Lookout,Hotel,Restaurant,Motel,Fast Food Restaurant,Bistro
5,Çamlıdere,40.49071,32.47463,1,Plaza,Forest,Art Museum,Turkish Restaurant,French Restaurant,Market,BBQ Joint,Track Stadium,Bistro,Gym Pool
6,Çankaya,39.92109,32.85391,1,Bookstore,Café,Coffee Shop,Turkish Restaurant,Sporting Goods Shop,Sandwich Place,Hotel,Pub,Gym,Kebab Restaurant
8,Elmadağ,39.91766,33.23444,1,Café,Park,Campground,Dessert Shop,Convenience Store,Kebab Restaurant,Bar,Big Box Store,Seafood Restaurant,Bus Stop
10,Evren,39.02225,33.80684,1,Park,River,Waterfront,Grocery Store,Steakhouse,Farmers Market,Entertainment Service,Event Space,Farm,Fish & Chips Shop
11,Gölbaşı,39.78973,32.80498,1,Café,Turkish Restaurant,Park,Restaurant,Breakfast Spot,Coffee Shop,Arcade,Middle Eastern Restaurant,Dessert Shop,Bakery


### Cluster2

In [44]:
ankara_merged.loc[ankara_merged['Cluster Labels'] == 2, ankara_merged.columns[[0] + list(range(4, ankara_merged.shape[1]))]]

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Etimesgut,39.9471,32.66075,2,Turkish Restaurant,Café,Restaurant,Steakhouse,Shopping Mall,Arcade,Dessert Shop,Park,Bakery,Snack Place


### Cluster3

In [45]:

ankara_merged.loc[ankara_merged['Cluster Labels'] == 3, ankara_merged.columns[[0] + list(range(5, ankara_merged.shape[1]))]]

,District,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## Report

### Introduction

This project is for who is planning to open a Coffee House with fast food in Ankara, Turkey. This project suggests the best locations for Coffee Houses in Ankara. Ankara is the capital of Turkey with a population of 5M. Turkish culture has deep roots in coffee. Also, it is getting popular because of its affordable prices. Coffee consumption has increased 13% and average consumption is 1.1 kg/person in 2018. This report explores which neighborhoods of Ankara have the most as well as the best Coffee Houses. Also, this project answers the questions “Where should I open an Coffee House?” and “Where should I stay If I want a tasty coffee?”

### Problem Description:
A Cofee restaurant is a business which prepares and serves fastfood and  drink to customers in return for money, either paid before the meal, after the meal, or with an open account. The City of Ankara is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.

Central and Turkeys immigrants, especially Europians Turists -wurst tost, bagels, cheesecake, hot dog, knishes, and delicatessens
Italian turists - Ankara-style pizza and Italian cuisine
Chinese and other Asian restaurants, sandwich joints, trattorias, diners, and coffeehouses are ubiquitous throughout the city
Middle Eastern foods such as falafel and kebabs examples of modern Ankara street food
It is famous for not just Cafe's but also for fine dining  restaurants.The city is home to "nearly one thousand of the finest and most diverse haute cuisine restaurants in the world".

### Data
•
District of Ankara are obtained from  
https://en.wikipedia.org/wiki/Ankara_Province[1]  
• Latitude and Longitude values are obtained by using "geocoder".  
• All data related to locations are obtained by using Foursquare API and Python  
Libraries. [2]

### Methodology
Master data which includes “District”, “Latitude” and “Longitude” information of Ankara.
Python Folium library was used to visualize map of Ankara and its districts. In order to get
Latitude and Longitudes of Ankara, geopy and geocoder are used. The map below was obtained.  
In order to explore and categorize places Foursquare API was used. Limit was set to 100 and
Radius was set to 500. Here is a head of the list Venues name, category, latitude and longitude
information from Forsquare API. 15 venues are returned.  

There are common venues in districts, so unsupervised K-means algorithm was used to cluster  
districts. K-Means cluster was set to 3 clusters, to reach optimum accuracy.  
Cluster 0: “Cafe” and “Plaza”  
Cluster 1: “Café” “Restaurant” and “Park”  
Cluster 2: “Turkish Restaurant” and “Bistro”  


### Discussion and Conclusion
Kmeans algorithm was used as part of this clustering study. K value was set to 3.   
For more
detailed and accurate guidance, the data set can be expanded, and the details of the
neighbourhood or street can also be drilled.    
It is obvious that Cluster 1 is the most viable clusters to open a new coffee house. Especially
Cluster 1 districts which have most common venues not Café such as Ayaş and Akyurt.
References  
[1] https://en.wikipedia.org/wiki/Ankara_Province
[2] https://developer.foursquare.com/